In [1]:
# !pip install z3

In [1]:
!pip install z3-solver


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()
# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")

https://langchain-ai.github.io/langgraph/concepts/low_level/

In [68]:
from typing import TypedDict, List, Dict, Optional, Union
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, SystemMessage
from langgraph.graph import StateGraph, END, START
from operator import add
from typing import Annotated
from langchain_openai import ChatOpenAI
import os

# LLM 초기화
llm = ChatOpenAI(
    temperature=0,
    model="gpt-4o"
)

# 1. 상태 정의
class LogicalAnalysisState(TypedDict):
    # 입력 텍스트
    input_text: str
    # 파악된 논리적 구문들
    logical_statements: List[str]
    # 각 구문의 진위 여부
    statement_validities: List[Dict[str, bool]]
    # 전체 신빙성 점수 (0-100)
    credibility_score: float
    # 분석 메시지들
    messages: Annotated[List[BaseMessage], add]

def analyze_context(state: LogicalAnalysisState) -> Dict:
    # 1단계: 문제 분석
    analysis_messages = [
        SystemMessage(content="""
            주어진 논리 문제를 분석하여 다음 정보를 추출하세요:
            
            1. 필요한 속성 목록:
               예시:
               - Logical (논리적)
               - Intuitive (직관적)
               - Math (수학적)
               등 문제에서 사용되는 모든 속성을 나열
            
            2. 논리적 관계 유형:
               - 단순 사실 ("X는 Y이다")
               - 조건부 관계 ("A이면 B이다")
               - 양방향 함의 ("A만이 B할 수 있다")
               - 복합 조건 ("A이고 B이면 C이다")
            
            3. 결론 명시
            
            분석 결과를 다음 형식으로 반환하세요:
            ---
            필요한 속성들:
            - 속성1 (설명)
            - 속성2 (설명)
            ...
            
            논리적 관계들:
            1. [단순 사실] X는 Y이다
            2. [조건부] A이면 B이다
            ...
            
            결론:
            Z는 W이다
            ---
        """),
        HumanMessage(content=f"다음 논리 문제를 분석해주세요:\n\n{state['input_text']}")
    ]
    
    # 분석 결과 얻기
    analysis_result = llm.invoke(analysis_messages)
    
    # 2단계: z3-solver 코드 생성
    code_messages = [
        SystemMessage(content="""
            분석된 논리 문제를 바탕으로 z3-solver 코드를 생성하세요.
            
            1. 변수 선언:
               - 분석된 모든 속성을 Bool 변수로 선언
               - 개별 인물의 변수는 생성하지 않음
            
            2. 관계 표현:
               - 단순 사실: solver.add(Y)
               - 조건부: solver.add(Implies(A, B))
               - 양방향 함의: solver.add(A == B)
               - 복합 조건: solver.add(Implies(And(A, B), C))
            
            3. 기본 논리 법칙 포함
            4. 결론의 부정 추가
            5. 검증 및 결과 출력
            
            다음 형식으로 코드를 생성하세요:
            ```python
            from z3 import *
            
            # Solver 초기화
            solver = Solver()
            
            # 변수 선언 - 모든 속성 포함
            A = Bool('A')  # 설명
            B = Bool('B')  # 설명
            ...
            
            # 기본 논리 법칙
            solver.add(Not(And(A, Not(A))))  # 모순율
            solver.add(Or(A, Not(A)))        # 배중율
            
            # 전제 추가
            solver.add(...)  # 각 전제
            
            # 결론의 부정
            solver.add(Not(...))
            
            # 검증 및 결과 출력
            result = solver.check()
            print(f"검증 결과: {result}")
            
            if result == unsat:
                print("논리적으로 타당합니다.")
                print("\\n논리적 설명:")
                print("1. 전제들로부터 결론이 필연적으로 도출됩니다.")
                print("2. 결론의 부정과 전제들이 동시에 참일 수 없습니다.")
                credibility = 100.0
            else:
                m = solver.model()
                print("논리적으로 타당하지 않습니다.")
                print("\\n반례 설명:")
                print(f"다음과 같은 경우가 가능합니다: {m}")
                print("\\n이 반례가 가능한 이유:")
                print("1. 이 경우 모든 전제는 참이지만")
                print("2. 결론은 거짓이 될 수 있습니다.")
                print("3. 따라서 전제들로부터 결론이 필연적으로 도출되지 않습니다.")
                credibility = 0.0

            print(f"\\n신빙성 점수: {credibility}")
            ```
        """),
        HumanMessage(content=f"분석 결과를 바탕으로 z3-solver 코드를 생성해주세요:\n\n{analysis_result.content}")
    ]
    
    # LLM 호출
    response = llm.invoke(code_messages)
    
    # 실행 가능한 코드 블록만 추출
    code_blocks = [block.strip() for block in response.content.split('```python')]
    if len(code_blocks) > 1:
        logical_statements = [code_blocks[1].strip('`').strip()]
    else:
        logical_statements = [response.content.strip()]
    
    return {
        "logical_statements": logical_statements,
        "messages": [
            HumanMessage(content=state["input_text"]),
            AIMessage(content=f"생성된 z3-solver 코드:\n{chr(10).join(logical_statements)}")
        ]
    }
    
def validate_statements(state: LogicalAnalysisState) -> Dict:
    """각 논리적 구문의 진위 여부 평가"""
    validities = []
    total_score = 0.0
    
    try:
        # 전체 코드를 하나의 블록으로 실행
        local_vars = {}
        exec("from z3 import *", local_vars)
        
        # 모든 구문을 하나로 합침
        full_code = "\n".join(state["logical_statements"])
        exec(full_code, local_vars)
        
        # solver 객체 확인
        solver = local_vars.get('solver')
        if solver is None:
            raise ValueError("No solver object found")
            
        # 결과 확인
        check_result = solver.check()
        is_valid = check_result == local_vars['unsat']
        
        if is_valid:
            reason = "논리식이 타당합니다 (결론의 부정이 모순을 일으킴)"
            total_score = 100.0
        else:
            reason = "논리식이 타당하지 않습니다"
            total_score = 0.0
            
        validities.append({
            "statement": full_code,
            "is_valid": is_valid,
            "reason": reason
        })
        
    except Exception as e:
        validities.append({
            "statement": full_code if 'full_code' in locals() else str(state["logical_statements"]),
            "is_valid": False,
            "reason": f"코드 실행 중 오류 발생: {str(e)}"
        })
        total_score = 0.0
    
    return {
        "statement_validities": validities,
        "credibility_score": total_score,
        "messages": [AIMessage(content=f"논리 구문 평가 완료: 신빙성 점수 {total_score:.1f}/100")]
    }

def calculate_credibility(state: LogicalAnalysisState) -> Dict:
    """논리적 타당성을 기반으로 신빙성 점수 계산"""
    validities = state.get("statement_validities", [])
    
    # 각 구문의 유효성 확인
    valid_count = sum(1 for v in validities if v.get("is_valid", False))
    total_count = len(validities)
    
    # 신빙성 점수 계산 (0-100)
    if total_count > 0:
        # unsat일 때 논리적으로 타당하므로, solver.check() == unsat인 경우를 확인
        for validity in validities:
            if "solver.check() == unsat" in validity["statement"]:
                weighted_score = 100.0 if validity["is_valid"] else 0.0
                break
        else:
            weighted_score = (valid_count / total_count) * 100.0
    else:
        weighted_score = 0.0
    
    return {
        "credibility_score": weighted_score,
        "messages": [AIMessage(content=f"전체 신빙성 점수: {weighted_score:.1f}/100")]
    }

# 3. 그래프 구성
def create_analysis_graph():
    """논리 분석 그래프 생성"""
    graph = StateGraph(LogicalAnalysisState)
    
    # 노드 추가
    graph.add_node("context_analyzer", analyze_context)
    graph.add_node("statement_validator", validate_statements)
    graph.add_node("credibility_calculator", calculate_credibility)
    
    # 엣지 추가
    graph.add_edge(START, "context_analyzer")
    graph.add_edge("context_analyzer", "statement_validator")
    graph.add_edge("statement_validator", "credibility_calculator")
    graph.add_edge("credibility_calculator", END)
    
    return graph.compile()

def analyze_text_logic(text: str) -> Dict:
    """텍스트의 논리성 분석 실행"""
    graph = create_analysis_graph()
    
    # 초기 상태
    initial_state = {
        "input_text": text,
        "logical_statements": [],
        "statement_validities": [],
        "credibility_score": 0.0,
        "messages": []
    }
    
    # 그래프 실행
    result = graph.invoke(initial_state)
    
    return {
        "credibility_score": result["credibility_score"],
        "logical_statements": result["logical_statements"],
        "validities": result["statement_validities"],
        "analysis_log": [m.content for m in result["messages"]]
    }

# 실행 예시
if __name__ == "__main__":
    test_text = """
"시간과 공간의 퍼즐"
전제들:

모든 존재는 시간적이거나 비시간적이다
시간적 존재가 가변적이면, 그것은 유한하다
비시간적 존재가 불변적이면, 그것은 무한하다
모든 존재는 공간적이거나 비공간적이다
공간적 존재가 연장적이면, 그것은 물질적이다
비공간적 존재가 비연장적이면, 그것은 정신적이다
유한한 존재는 물질적이거나 정신적이다
무한한 존재는 초월적이거나 내재적이다
물질적 존재가 초월적이면, 그것은 역설적이다
정신적 존재가 내재적이면, 그것은 역설적이다
역설적 존재는 모순적이거나 변증법적이다
X는 존재이다
X는 시간적이고 공간적이다
X는 가변적이고 연장적이다
X는 초월적이다
모순적인 존재는 불가능하다
변증법적 존재만이 가능하다

결론: X는 변증법적이다
    """
    
    result = analyze_text_logic(test_text)
    
    print(f"신빙성 점수: {result['credibility_score']:.1f}/100")
    print("\n논리 구문들:")
    for stmt in result["logical_statements"]:
        print(f"- {stmt}")
    # print("\n분석 로그:")
    # for log in result["analysis_log"]:
    #     print(f"- {log}")

검증 결과: unsat
논리적으로 타당합니다.

논리적 설명:
1. 전제들로부터 결론이 필연적으로 도출됩니다.
2. 결론의 부정과 전제들이 동시에 참일 수 없습니다.

신빙성 점수: 100.0
신빙성 점수: 100.0/100

논리 구문들:
- from z3 import *

# Solver 초기화
solver = Solver()

# 변수 선언 - 모든 속성 포함
시간적 = Bool('시간적')  # 시간과 관련된 속성
비시간적 = Bool('비시간적')  # 시간과 관련이 없는 속성
가변적 = Bool('가변적')  # 변화할 수 있는 속성
유한 = Bool('유한')  # 한계가 있는 속성
불변적 = Bool('불변적')  # 변화하지 않는 속성
무한 = Bool('무한')  # 한계가 없는 속성
공간적 = Bool('공간적')  # 공간과 관련된 속성
비공간적 = Bool('비공간적')  # 공간과 관련이 없는 속성
연장적 = Bool('연장적')  # 공간적으로 확장된 속성
물질적 = Bool('물질적')  # 물질로 이루어진 속성
비연장적 = Bool('비연장적')  # 공간적으로 확장되지 않은 속성
정신적 = Bool('정신적')  # 정신과 관련된 속성
초월적 = Bool('초월적')  # 초월적인 속성
내재적 = Bool('내재적')  # 내재적인 속성
역설적 = Bool('역설적')  # 모순을 포함하는 속성
모순적 = Bool('모순적')  # 논리적으로 불가능한 속성
변증법적 = Bool('변증법적')  # 논리적으로 가능한 속성

# 기본 논리 법칙
solver.add(Not(And(시간적, Not(시간적))))  # 모순율
solver.add(Or(시간적, Not(시간적)))        # 배중율

# 전제 추가
solver.add(Or(시간적, 비시간적))  # 모든 존재는 시간적이거나 비시간적이다
solver.add(Implies(And(시간적, 가변적), 유한))  # 시간적 존재가 가변적이면, 그것은 유한하다
solver.ad

In [ ]:
probelm1  = """
  "시간과 공간의 퍼즐"
전제들:

모든 존재는 시간적이거나 비시간적이다
시간적 존재가 가변적이면, 그것은 유한하다
비시간적 존재가 불변적이면, 그것은 무한하다
모든 존재는 공간적이거나 비공간적이다
공간적 존재가 연장적이면, 그것은 물질적이다
비공간적 존재가 비연장적이면, 그것은 정신적이다
유한한 존재는 물질적이거나 정신적이다
무한한 존재는 초월적이거나 내재적이다
물질적 존재가 초월적이면, 그것은 역설적이다
정신적 존재가 내재적이면, 그것은 역설적이다
역설적 존재는 모순적이거나 변증법적이다
X는 존재이다
X는 시간적이고 공간적이다
X는 가변적이고 연장적이다
X는 초월적이다
모순적인 존재는 불가능하다
변증법적 존재만이 가능하다

결론: X는 변증법적이다
  """

In [82]:
from smolagents import CodeAgent, LiteLLMModel

# Initialize the model with OpenAI settings
model = LiteLLMModel(model_id="gpt-4o")


# Create the CodeAgent with the specified model
agent = CodeAgent(tools=[], model=model, add_base_tools=True)


# Run the agent with the query
response = agent.run(
   HumanMessage(content="aa").content,
)


print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ aa                                                                                                              │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o ─────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─ Executing this code: ──────────────────────────────────────────────────────────────────────────────────────────╮
│   1 final_answer("The input 'aa' is unclear. Please provide a specific task or question for me to assist you.") │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Out - Final answer: The input 'aa' is unclear. Please provide a specific task or question for me to assist you.

[Step 0: Duration 1.63 seconds| Input tokens: 2,068 | Output tokens: 76]

The input 'aa' is unclear. Please provide a specific task or question for me to assist you.


In [81]:
from smolagents import CodeAgent, LiteLLMModel

# Initialize the model with OpenAI settings
model = LiteLLMModel(model_id="gpt-4o")


# Create the CodeAgent with the specified model
agent = CodeAgent(tools=[], model=model, add_base_tools=True)


# Run the agent with the query
response = agent.run(
   "Could you give me the 118th number in the Fibonacci sequence?",
)


print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you give me the 118th number in the Fibonacci sequence?                                                   │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o ─────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─ Executing this code: ──────────────────────────────────────────────────────────────────────────────────────────╮
│   1 def fibonacci(n):                                                                                           │
│   2     a, b = 0, 1                                                                                             │
│   3     for _ in range(n - 1):                                                                                  │
│   4         a, b = b, a + b                                                                                     │
│   5     return a                                                                                                │
│   6                                                                                                             │
│   7 fibonacci_118th_number = fibonacci(118)                                                                     │
│   8 final_answer(fibonacci_118th_number)                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Out - Final answer: 1264937032042997393488322

[Step 0: Duration 3.85 seconds| Input tokens: 2,081 | Output tokens: 118]

1264937032042997393488322


In [85]:
from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage


# Define state schema
class AgentState(dict):
   input: str
   response: str


# Initialize components
workflow = StateGraph(AgentState)


# Replace with your actual OpenAI API key
llm = ChatOpenAI(model="gpt-4o", temperature=0, api_key="sk-api_key")


# Define nodes
def generate_response(state):
   result = agent.run(HumanMessage(content=state["input"]).content)
   return {"response": result}


# Set up workflow
workflow.add_node("fibonacci_solver", generate_response)
workflow.set_entry_point("fibonacci_solver")
workflow.set_finish_point("fibonacci_solver")


# Execute workflow
app = workflow.compile()
result = app.invoke({"input": "Calculate the 118th Fibonacci number"})
print("LangGraph Result:", result["response"])

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Calculate the 118th Fibonacci number                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o ─────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─ Executing this code: ──────────────────────────────────────────────────────────────────────────────────────────╮
│   1 def fibonacci(n):                                                                                           │
│   2     a, b = 0, 1                                                                                             │
│   3     for _ in range(n - 1):                                                                                  │
│   4         a, b = b, a + b                                                                                     │
│   5     return b                                                                                                │
│   6                                                                                                             │
│   7 fibonacci_number_118 = fibonacci(118)                                                                       │
│   8 final_answer(fibonacci_number_118)                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Out - Final answer: 2046711111473984623691759

[Step 0: Duration 2.18 seconds| Input tokens: 2,074 | Output tokens: 107]

AttributeError: 'int' object has no attribute 'content'